# Sentiwordnet avec POS et synsets de wordnet
##### zero negatif detecté 


In [34]:
import pandas as pd     
import numpy as np

In [3]:
df = pd.read_pickle('df_model.p')
df_tweet_en_neutral = df[df['sentiment'] == 'neutral'].sample(n=100)
#df_tweet_en_neutral
df_tweet_en_positive = df[df['sentiment'] == 'positive'].sample(n=100)
#df_tweet_en_positive
df_tweet_en_negative = df[df['sentiment'] == 'negative'].sample(n=100)
#df_tweet_en_negative

In [4]:
dff = pd.concat([df_tweet_en_neutral,df_tweet_en_positive])
dff = pd.concat([dff,df_tweet_en_negative])

In [23]:
#dff['clean_text']
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
import nltk
nltk.download('punkt') 
nltk.download('averaged_perceptron_tagger')
nltk.download('sentiwordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\smail\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\smail\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\smail\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\sentiwordnet.zip.


True

In [25]:
 
lemmatizer = WordNetLemmatizer()
 
 
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None
 
 
def clean_text(text):
    #text = text.decode("utf-8")
 
    return text
 
 
def swn_polarity(text):
    """
    Return a sentiment polarity: 0 = negative, 1 = positive
    """
 
    sentiment = 0.0
    tokens_count = 0
 
    text = clean_text(text)
 
 
    raw_sentences = sent_tokenize(text)
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
 
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
 
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
 
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
 
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
 
    # judgment call ? Default to positive or negative
    if not tokens_count:
        return 0
 
    # sum greater than 0 => positive sentiment
    if sentiment >= 0:
        return 1
 
    # negative sentiment
    return 0

In [32]:
Y_pred = []

for sentence in dff['clean_text']:
    vs = swn_polarity(sentence)
    Y_pred.append(vs)
    print(vs)

0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
0
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
0
1
0
1
1
0
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
0
0
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
0
1
0
0
1
1
0
1
0
1
0
1
0
1
1
1
0
1
1
1
1
0
1
0
0
1
0
0
0
0
1
1
1
0
0
1
0
1
1
0
1
1
1
1
1
1
1
0
0
1
1
1
0
0
1
1
0
1
1
1
1
0
1
1
0
1
1
1
1
0
0
1
0
1
1


In [29]:
dff['sentiment'][dff['sentiment']=='neutral'] = 0
dff['sentiment'][dff['sentiment']=='positive'] = 1
dff['sentiment'][dff['sentiment']=='negative'] = -1

C:\Users\smail\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\smail\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\smail\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
Y = dff['sentiment'].values
Y.shape

(300,)

In [35]:
Y_pred = np.asarray(Y_pred)

In [36]:
Y = [x for x in Y]

In [37]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [38]:
cm = confusion_matrix(Y , Y_pred, labels=[1,-1,0])
acc = accuracy_score(Y, Y_pred)

In [39]:
print(cm)
print(acc)

[[82  0 18]
 [67  0 33]
 [85  0 15]]
0.3233333333333333


# References
* https://nlpforhackers.io/sentiment-analysis-intro/ 